# **Configuración Maestra (Instalación + Vinculación a Drive)**

In [ ]:
import os
import shutil
from google.colab import drive

# 1. Conectar Google Drive
drive.mount('/content/drive')

# 2. Definir Rutas
base_path = "/content"
comfy_path = f"{base_path}/ComfyUI"
# Esta será la carpeta raíz en tu Drive para guardar tus cosas
GDRIVE_BASE = '/content/drive/MyDrive/ComfyUI_Data'

# 3. Instalar Herramientas de Aceleración (Aria2)
!apt-get -y install -qq aria2

# 4. Instalar ComfyUI en disco local de Colab (Efímero - Máxima Velocidad)
if not os.path.exists(comfy_path):
    print("⚡ Instalando ComfyUI en el disco local de Colab...")
    %cd {base_path}
    !git clone https://github.com/comfyanonymous/ComfyUI
    %cd {comfy_path}
    !pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121

    # Instalar Manager y soporte GGUF (Necesario para Flux)
    %cd custom_nodes
    !git clone https://github.com/ltdrdata/ComfyUI-Manager
    !git clone https://github.com/city96/ComfyUI-GGUF
    !pip install -r ComfyUI-GGUF/requirements.txt
    %cd ..
else:
    print("✅ ComfyUI ya está instalado.")

# 5. CONFIGURACIÓN DE ENLACES (La parte inteligente)
# Esto conecta las carpetas de Colab a tu Drive solo para lo importante.

LINKS = {
    'output': f'{GDRIVE_BASE}/output',      # Tus imágenes generadas
    'input':  f'{GDRIVE_BASE}/input',       # Imágenes que subas para editar
    'user':   f'{GDRIVE_BASE}/user'         # Tus Workflows guardados y config
}

# Asegurar que la base en Drive exista
if not os.path.exists(GDRIVE_BASE):
    os.makedirs(GDRIVE_BASE, exist_ok=True)

print("🔄 Configurando enlaces simbólicos inteligentes...")

for name, target in LINKS.items():
    source = os.path.join(comfy_path, name)

    # A. Asegurar que la carpeta destino exista en Drive
    if not os.path.exists(target):
        os.makedirs(target, exist_ok=True)
        print(f"   📂 Creada carpeta en Drive: {target}")

    # B. Eliminar la carpeta local actual (para reemplazarla por el enlace)
    if os.path.exists(source):
        if os.path.islink(source):
            os.unlink(source) # Borrar si ya era un enlace
        elif os.path.isdir(source):
            shutil.rmtree(source) # Borrar si era una carpeta normal

    # C. Crear el enlace simbólico (Puente)
    os.symlink(target, source)
    print(f"   🔗 Vinculado: {name} -> Drive")

print("\n✅ ¡Listo! Sistema Híbrido Activo.")

# **Gestor de Descargas (Modelos Flux + GGUF)**

In [ ]:
import os

# ==============================================================================
# 📋 LISTA MAESTRA DE MODELOS (ENLACES ARREGLADOS)
# ==============================================================================

DOWNLOAD_LIST = [


  ("https://huggingface.co/Comfy-Org/flux1-schnell/resolve/main/flux1-schnell-fp8.safetensors?download=true", "flux1-schnell-fp8.safetensors", "checkpoints"),

  ("https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn.safetensors?download=true", "t5xxl_fp8_e4m3fn.safetensors", "clip"),
  ("https://huggingface.co/zer0int/CLIP-GmP-ViT-L-14/resolve/main/ViT-L-14-TEXT-detail-improved-hiT-GmP-HF.safetensors?download=true", "ViT-L-14-TEXT-detail-improved-hiT-GmP-HF.safetensors", "clip"),

  ("https://huggingface.co/ffxvs/vae-flux/blob/main/ae.safetensors", "ae.safetensors", "vae"),

  ("https://civitai.com/api/download/models/1153358?type=Model&format=SafeTensor&size=full&fp=fp8", "agfluxSchnell_realistic23.safetensors", "diffusion_models"),

  ("https://civitai.com/api/download/models/1081450?type=Model&format=SafeTensor", "skin texture style v5.safetensors", "lora"),
  ("https://civitai.com/api/download/models/918612?type=Model&format=SafeTensor", "REALISM.safetensors", "lora"),
  ("https://civitai.com/api/download/models/804967?type=Model&format=SafeTensor", "Hand v2.safetensors", "lora"),

]

# ==============================================================================
# ⚙️ MOTOR DE DESCARGA
# ==============================================================================
print("🚀 Iniciando Gestor de Descargas...")

base_path = "/content/ComfyUI/models"

# HE AGREGADO LAS CARPETAS QUE FALTABAN AQUÍ ABAJO 👇
folders = {
    "unet": f"{base_path}/unet",
    "clip": f"{base_path}/clip",
    "vae": f"{base_path}/vae",
    "lora": f"{base_path}/loras",
    "loras": f"{base_path}/loras",  # Alias por si escribes 'loras' en plural
    "checkpoint": f"{base_path}/checkpoints", # Alias por si escribes 'checkpoints' en plural
    "checkpoints": f"{base_path}/checkpoints",
    "controlnet": f"{base_path}/controlnet",
    "clip_vision": f"{base_path}/clip_vision",        # NUEVO
    "diffusion_models": f"{base_path}/diffusion_models", # NUEVO (Para Wan/Chrono)
    "text_encoders": f"{base_path}/text_encoders"     # NUEVO (ComfyUI usa esta carpeta a veces)
}

# Crear todas las carpetas
for path in folders.values():
    os.makedirs(path, exist_ok=True)

for url, filename, type_key in DOWNLOAD_LIST:
    # Ahora sí encontrará las claves nuevas
    if type_key not in folders:
        print(f"⚠️ Error: No existe la categoría '{type_key}' en el diccionario de carpetas.")
        continue

    destination = folders[type_key]
    file_path = os.path.join(destination, filename)

    if os.path.exists(file_path):
        print(f"✅ Ya existe: {filename}")
    else:
        print(f"⬇️ Descargando: {filename} en [{type_key}]...")
        # Usamos user-agent para evitar bloqueos en archivos grandes
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M --user-agent="Mozilla/5.0" "{url}" -d "{destination}" -o "{filename}"

print("\n✨ ¡Descargas corregidas y completadas!")

# **Ejecución (Pinggy + ComfyUI)**

In [ ]:
import os

print("💾 Creando archivo de intercambio (SWAP) para aumentar la RAM del sistema...")

# 1. Verificar si ya existe swap
if os.path.exists('/swapfile'):
    print("✅ El archivo Swap ya existe.")
else:
    # 2. Crear un archivo de 8GB en el disco
    # Usamos 'fallocate' que es más rápido, o dd si falla
    !fallocate -l 8G /swapfile
    !chmod 600 /swapfile
    !mkswap /swapfile
    !swapon /swapfile
    print("✅ Swap de 8GB creado y activado.")

# 3. Verificar memoria total ahora
print("\n📊 Memoria Total Disponible (RAM + SWAP):")
!free -h

In [ ]:
import subprocess
import threading
import time
import socket
import os
import gc
import torch

# ==========================================
# 1. LIMPIEZA AGRESIVA
# ==========================================
print("🧹 Matando procesos y limpiando memoria...")
!pkill -9 -f "python main.py"
!pkill -9 -f cloudflared
!pkill -9 -f ngrok
!pkill -9 ssh
gc.collect() # Limpieza de basura de Python
torch.cuda.empty_cache() # Limpieza de GPU
time.sleep(2)

# ==========================================
# 2. CREAR SUPER-SWAP (12 GB)
# ==========================================
# Esto usará 12GB de tu disco duro como si fuera RAM lenta.
# Si la RAM real se llena, usará esto en lugar de cerrar el programa.
swap_file = '/swapfile'

if os.path.exists(swap_file):
    # Si ya existe, verificamos si es del tamaño correcto, si no, lo borramos
    size = os.path.getsize(swap_file)
    if size < 12 * 1024 * 1024 * 1024: # Si es menor a 12GB
        !swapoff {swap_file}
        !rm {swap_file}
        print("⚠️ Swap anterior muy pequeño. Creando uno nuevo de 12GB...")

if not os.path.exists(swap_file):
    print("💾 Creando archivo de intercambio de 12GB (Esto tardará unos segundos)...")
    # Usamos fallocate para reservar espacio
    !fallocate -l 12G {swap_file}
    !chmod 600 {swap_file}
    !mkswap {swap_file}
    !swapon {swap_file}
    print("✅ Swap de 12GB Activado.")
else:
    print("✅ Swap de 12GB ya estaba activo.")

# Verificamos memoria
print("\n📊 ESTADO DE MEMORIA BLINDADA:")
!free -h

# ==========================================
# 3. TÚNEL PINGGY
# ==========================================
def pinggy_tunnel(port):
    print(f"Intentando abrir túnel en el puerto {port}...")
    cmd = f"ssh -p 443 -R0:localhost:{port} -o StrictHostKeyChecking=no -o ServerAliveInterval=30 a.pinggy.io"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in process.stdout:
        l = line.decode().strip()
        if "pinggy.link" in l:
            print("\n" + "★"*50)
            print("✅ ¡TÚNEL ESTABLE GENERADO!")
            print(f"👉 ENTRA AQUÍ: {l}")
            print("★"*50 + "\n")

threading.Thread(target=pinggy_tunnel, daemon=True, args=(8188,)).start()

# ==========================================
# 4. INICIO DE COMFYUI (MODO AHORRO TOTAL)
# ==========================================
%cd /content/ComfyUI
print("🐢 Iniciando ComfyUI en MODO TANQUE (Estabilidad máxima, velocidad media)...")

# --disable-smart-memory: Evita que ComfyUI intente optimizar guardando cosas en RAM.
# --lowvram: Divide el trabajo de la GPU.
!python main.py --dont-print-server --listen 0.0.0.0 --port 8188 --enable-cors-header "*" --preview-method auto --lowvram --disable-smart-memory